接下来我们用keras来实现AlexNet，使用的数据集是cifa-10，cifa-10这个数据集共有60000张彩色图像，每张图像都是32×32，分为10个类，每类6000张图，也就是说这是一个10分类的图像识别问题。但AlexNet的输入图像大小为227×227，如果采用一样的参数在我们的数据集上是行不通的，其次本节的关键点还是要让读者掌握AlexNet的优点，因此此处我们将对卷积核做一些简单的修改，把卷积核的大小都改为3×3，并减少卷积核的数量，池化层大小都改为3×3。当然，有兴趣的读者可以自行下载ImageNet数据来对原本AlexNet进行测试，这里为大家提供下载地址：
ImageNet LSVRC 2012（147.90G）：http://academictorrents.com/details/a306397ccf9c2ead27155983c254227c0fd938e2

首先读取我们的数据集，这里我们对数据做一个归一化的处理，方便模型收敛，并且对label做一个one-hot处理，因为是10分类问题，所以num_classes这个参数的值为10。

In [16]:
from keras.datasets import cifar10
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

模型搭建阶段我们依旧采用序贯模型Sequential，整体的结构和上文有略微差别，主要在于卷积核大小和池化层大小。

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical

model = Sequential()
model.add(Conv2D(96, (3, 3), input_shape=(32, 32, 3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 30, 30, 96)        2688      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 96)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 15, 15, 256)       221440    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 5, 5, 256)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 5, 5, 384)         885120    
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 5, 5, 384)         1327488   
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 5, 5, 256)         884992    
__________

定义损失函数、优化方法、评估指标。

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

模型准备完成后就可以开始训练了，这里我们引入了EarlyStopping，EarlyStopping可以保证模型在准确率不在提升的前提下提前结束训练，其中monitor参数表示的是停止时参考的指标，这里我们采用的是准确率，patience的意思是多少轮指标没有改变模型训练就停止，这里我们使用的是5，也就是说如果训练五轮，准确率依旧没有提升即停止训练。

In [22]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=5, verbose=1)
model.fit(x_train, y_train, batch_size=64,epochs=200, callbacks=[early_stopping])

Epoch 1/200
50000/50000 [==============================] - 35s 691us/step - loss: 0.1214 - acc: 0.9675
Epoch 2/200
50000/50000 [==============================] - 34s 680us/step - loss: 0.1292 - acc: 0.9659
Epoch 3/200
50000/50000 [==============================] - 34s 682us/step - loss: 0.1189 - acc: 0.9684
Epoch 4/200
50000/50000 [==============================] - 34s 683us/step - loss: 0.1233 - acc: 0.9658
Epoch 5/200
50000/50000 [==============================] - ETA: 0s - loss: 0.1184 - acc: 0.969 - 34s 684us/step - loss: 0.1186 - acc: 0.9694
Epoch 6/200
50000/50000 [==============================] - 34s 684us/step - loss: 0.1234 - acc: 0.9671
Epoch 7/200
50000/50000 [==============================] - 34s 683us/step - loss: 0.1136 - acc: 0.9719
Epoch 8/200
50000/50000 [==============================] - 34s 683us/step - loss: 0.1234 - acc: 0.9680
Epoch 9/200
50000/50000 [==============================] - 34s 684us/step - loss: 0.1385 - acc: 0.9649
Epoch 10/200
50000/50000 [=========

In [23]:
score = model.evaluate(x_test, y_test, batch_size=64)
print('Test accuracy:', score[1])

10000/10000 [==============================] - 2s 219us/step
Test accuracy: 0.7427
